In [1]:
# Dependencies:

#!pip install transformers ipywidgets IPython

# If running on Colab:
#!git clone https://github.com/sam-paech/antislop-sampler.git
#!mv antislop-sampler/src .
#!mv antislop-sampler/slop_phrase_prob_adjustments.json .

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from src.antislop_generate import generate_antislop, chat_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to(device)
print('Model loaded')

Model loaded


In [3]:
import json
if os.path.exists('slop_phrase_prob_adjustments.json'):
    with open('slop_phrase_prob_adjustments.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:500])

In [4]:
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

In [10]:
# Chat generation with streaming
messages = [
    {"role": "user", "content": prompt}
]
tokens = []
text = ''
for token in chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_new_tokens=400,
    # Antislop sampling may be less reliable at low temperatures.
    temperature=1,    
    min_p=0.1,
    # The adjustment_strength param scales how strongly the probability adjustments are applied.
    # A value of 1 means the values in slop_phrase_prob_adjustments (or the defaults) are used unmodified.
    # Reasonable values are 0 (disabled) thru 100+ (effectively banning the list).
    adjustment_strength=100.0,
    # Optional: Provide a list of slop phrases and probability adjustments
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True,
    stream_smoothing=True, # On by default; this will smooth out the stutters from backtracking.
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

In the heart of Future City, where sleek skyscrapers and holographic advertisements seemed to stretch on forever, a lone figure stood out among the throngs of pedestrians. Her name was Aria, but the people of this futuristic city knew her as the Weaver of Tales.

Aria was a master of her craft, a weaver of extraordinary weavings that transported viewers to realms both familiar and unknown. Her studio, a small, cozy shop tucked away in a quiet alley, was a haven of warm colors and soft sounds, where the scent of wool and spices mingled with the hum of holographic machines.

As the sun began to set, casting a golden glow over the city, Aria would take her latest commission from the esteemed Guild of Storykeepers, a group of revered artisans responsible for preserving the city's rich cultural heritage. These esteemed weavers were tasked with creating breathtaking weavings that told the stories of the city's history, myths, and legends.

Tonight, Aria was working on a particularly special 

In [6]:
# Chat generation without streaming
messages = [
    {"role": "user", "content": prompt}
]
generated_text = chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_length=400,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)
print(tokenizer.decode(generated_text))

In the year 2254, in the sprawling city of New Eden, the sound of hammers ringing against steel and the hum of machinery filled the air. The city was a marvel of modern technology, where humans and AI had merged to create a utopia of efficiency and progress. But despite the wonders of this future world, there was one craftsman who stood out from the rest: a talented weaver named Eliara.

Eliara lived in a small, intricately carved wooden apartment in the heart of New Eden's prestigious Upper District. Her home was a sanctuary, filled with the soft glow of luminescent fibers that infused her creations with a soft, ethereal light. The walls were adorned with Eliara's stunning works of art – magnificent wall hangings that told stories of the past, present, and future.

As the sun set over New Eden, casting a warm orange glow over the city, Eliara set to work on her latest commission. She had been hired to create a grand mural for the grand ballroom of the city's grandest social club, the 

In [7]:
# generate without streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
generated_text = generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)        
print(tokenizer.decode(generated_text))

 15-year-olds are gathered around her workshop, mesmerized by the colors and patterns that dance across her loom.

The young apprentices, each with their own unique skillset and expertise, watch as their teacher expertly weaves together tales of old. The air is thick with the scent of wool and the soft hum of the loom, creating an enchanting atmosphere that is at once calming and exhilarating.

As the day wears on, the group's fascination grows, and they begin to ask questions about the secrets behind her enchanted loom. One young apprentice, a skilled weaver herself, asks the question that will change the course of their lives: "How do you weave the threads of time?" The question sparks a flurry of excited chatter and the young teacher smiles, knowing that the moment has arrived to reveal the true magic of her craft.

With a nod, she begins to weave a new tale, one that weaves together the threads of the past, present, and future, creating a breathtaking narrative that will capture th

In [11]:
# generate with streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
tokens = []
text = ""
for token in generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    adjustment_strength=100.0,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

 5 adventurers were hired for a mission to retrieve a rare and valuable artifact from the long-abandoned Spire of Eldrid.
The group consisted of:
- Arin the Brave, a skilled warrior with unparalleled combat prowess
- Lyra the Enigmatic, an expert in ancient lore and forgotten technologies
- Kael the Charismatic, a smooth-talking diplomat and negotiator
- Mira the Scholar, a brilliant and book-smart expert in the arcane
- and Zephyr the Wild, a fearless and agile hunter with unparalleled tracking skills.

Together, the group ventured into the treacherous Spire of Eldrid, a massive structure built by the ancient Eldridians to reach the uppermost level of their sprawling city. The Spire was said to contain a treasure trove of knowledge and artifacts, but it was also rumored to be cursed, filled with deadly traps and fearsome creatures.

As they reached the heart of the Spire, the group encountered a massive stone door adorned with ancient runes. Kael, with his quick wit and knowledge of a